In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time
import minimalmodbus


def connect_robot(ip="192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip)  # IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set


def initialize_decapper():
    instrument = minimalmodbus.Instrument(
        "/dev/ttyUSB0", 1
    )  # port name, slave address (in decimal)
    instrument.serial.baudrate = 115200  # Baud
    minimalmodbus.MODE_RTU = "rtu"
    instrument.write_register(256, 1)  # initialize gripper
    time.sleep(3)
    instrument.write_register(259, 1000)  # open/close gripper: 1=closed, 1000=open
    return instrument

In [2]:
rtde_c, rtde_r, rtde_io_set = connect_robot()
instrument = initialize_decapper()

In [3]:
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(0)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")

Activating Gripper
Gripper activated


In [4]:
home_pos = [
    -0.19171785391383447,
    0.13627313847997366,
    0.22530939672205824,
    -2.216400117559973,
    -2.214550992098179,
    -0.011655385945140848,
]

home_joints = [
    3.1417369842529297,
    -0.7853723925403138,
    -2.3560631275177,
    -1.570796628991598,
    1.5707793235778809,
    0.0006903879693709314,
]

turned_to_medium_joints = [
    1.2,
    -0.7853546303561707,
    -2.3560631275177,
    -1.5708304844298304,
    1.5707474946975708,
    -0.09389096895326787,
]

front_of_medium_joints = [
    1.9955487251281738,
    -1.7771555385985316,
    -2.4277005195617676,
    -2.0229040584959925,
    0.4453906714916229,
    -0.03315765062441045,
]

gripping_medium_pos = [
    0.32302227235079833,
    0.31220251479806876,
    0.04961699910373372,
    1.242464345380708,
    1.205263846451164,
    1.2070471275394923,
]

front_of_decapper_pos = [
    -0.3331419874539266,
    0.45369367425701873,
    0.11327199744738782,
    -0.5143837921752208,
    -1.9351717108776767,
    -1.9165440370569073,
]

below_decapper_pos = [
    -0.3316146877373861,
    0.5808466223324783,
    0.11332700814063355,
    -0.5354754656825224,
    -1.9199056078070442,
    -1.9018092257048145,
]

decapping_pos = [
    -0.3316520531694605,
    0.5816230354402923,
    0.13217691438575127,
    -0.5354853756905684,
    -1.9197885063172049,
    -1.9017210690212516,
]

In [5]:
def get_bottle():
    rtde_c.moveJ(home_joints, 0.5, 0.5)
    gripper.open()
    gripper.set_force(30)
    rtde_c.moveJ(turned_to_medium_joints, 0.5, 0.5)
    rtde_c.moveJ(front_of_medium_joints, 0.2, 0.2)
    rtde_c.moveL(gripping_medium_pos, 0.2, 0.2)
    gripper.close()
    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] += 0.2
    rtde_c.moveL(current_pos, 0.2, 0.2)
    current_pos = rtde_r.getActualTCPPose()
    current_pos[0] -= 0.43
    rtde_c.moveL(current_pos, 0.2, 0.2)
    current_joints = rtde_r.getActualQ()
    current_joints[4] -= 0.5
    rtde_c.moveJ(current_joints, 0.2, 0.2)
    current_pos = rtde_r.getActualTCPPose()
    current_pos[1] -= 0.1
    rtde_c.moveL(current_pos, 0.2, 0.2)
    current_joints = rtde_r.getActualQ()
    current_joints[4] -= 1.08
    rtde_c.moveJ(current_joints, 0.2, 0.2)


def decap_bottle():
    instrument.write_register(259, 1000)  # open gripper
    rtde_c.moveL(front_of_decapper_pos, 0.2, 0.2)
    rtde_c.moveL(below_decapper_pos, 0.2, 0.2)
    rtde_c.moveL(decapping_pos, 0.1, 0.1)
    time.sleep(2)
    instrument.write_register(259, 220)  # grip bottle
    time.sleep(1)
    instrument.write_register(261, 720)
    time.sleep(1)
    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] -= 0.025
    rtde_c.moveL(current_pos, 0.2, 0.2)
    rtde_c.moveL(front_of_decapper_pos, 0.2, 0.2)
    gripper.open()
    rtde_c.moveJ(home_joints, 0.5, 0.5)

In [15]:
get_bottle()

In [16]:
decap_bottle()

In [17]:
instrument.write_register(259, 1000)  # open gripper